In [1]:
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

In [12]:
validatedf = pd.read_csv("test.csv")
df = pd.read_csv("train.csv")

In [13]:
df= df.drop(['latitude','longitude', 'id', 'carNumber', 'date', 'time', 'race'], axis=1)
validatedf = validatedf.drop(['latitude','longitude', 'id', 'carNumber', 'date', 'time', 'race'], axis=1)

In [14]:
df.head(1)

,circuit,altitude,driver,constructor,avgDriverFinish,avgConstructorFinish,lapNumber,lapPosition,pitStop,pitCount,pitTime_ms,lapTime_ms
0,Silverstone Circuit,153,"Ricciardo, Daniel",Red Bull,9.799145,3.033426,18,10,0,0,0.0,94846


In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, mean_squared_error

# Assuming 'circuit', 'driver', and 'constructor' are categorical columns
df = pd.get_dummies(df, columns=['circuit', 'driver', 'constructor'])

# Extract features (X) and target variable (y)
X = df.drop(columns=['lapTime_ms'])
y = df['lapTime_ms']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the neural network model
def create_model(units=64):
    model = Sequential()
    model.add(Dense(units=units, activation='relu', input_dim=X_train.shape[1]))
    model.add(Dense(units=32, activation='relu'))
    model.add(Dense(units=1, activation='linear'))
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

# Set up the pipeline including preprocessing and the neural network model
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=create_model, epochs=50, batch_size=32, verbose=0))
])

# Set up hyperparameter tuning using GridSearchCV
param_grid = {
    'model__epochs': [50, 100],
    'model__batch_size': [32, 64],
    'model__units': [32, 64, 128]
}

# Define the scoring metric for GridSearchCV
scoring_metric = make_scorer(mean_squared_error, greater_is_better=False)

# Create GridSearchCV instance
grid_search = GridSearchCV(pipeline, param_grid=param_grid, scoring=scoring_metric, cv=5)

# Fit the model using GridSearchCV
grid_search.fit(X_train, y_train)

# Print the best hyperparameter combination
best_params = grid_search.best_params_
print("Best Hyperparameter Combination:")
print(best_params)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
# Evaluate the best model on the test set
mse = grid_search.score(X_test, y_test)
print(f'Mean Squared Error on Test Set: {mse}')

# Make predictions on new data (e.g., validatedf)
# X_valid_scaled = scaler.transform(X_valid)
# predicted_laptimes_valid = best_model.predict(X_valid_scaled)


C:\Users\ajed8\AppData\Local\Temp\ipykernel_15216\2427635866.py:31: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('model', tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=create_model, epochs=50, batch_size=32, verbose=0))


Best Hyperparameter Combination:
{'model__batch_size': 32, 'model__epochs': 100, 'model__units': 128}


TypeError: cannot unpack non-iterable float object

In [16]:
# Evaluate the best model on the test set
mse = grid_search.score(X_test, y_test)
print(f'Mean Squared Error on Test Set: {mse}')

Mean Squared Error on Test Set: -3943896930.3754907


In [20]:
# Make predictions on new data (e.g., validatedf)
validatedf = pd.read_csv("test.csv")
# Assuming 'circuit', 'driver', and 'constructor' are categorical columns in validatedf
validatedf_encoded = pd.get_dummies(validatedf, columns=['circuit', 'driver', 'constructor'])

# Ensure that the columns in validatedf match the columns used for training
validatedf_encoded = validatedf_encoded.reindex(columns=X_train.columns, fill_value=0)

# Standardize the features using the same scaler used for training
X_valid_scaled = scaler.transform(validatedf_encoded)

# Make predictions on new data (e.g., validatedf)
predicted_laptimes_valid = best_model.predict(X_valid_scaled)


C:\Users\ajed8\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [21]:
predicted_laptimes_valid

array([ 793182.94, 1027606.75,  970317.6 , ..., 1018328.94, 1117817.  ,
        939392.5 ], dtype=float32)